In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/spam-classification-for-basic-nlp/Spam Email raw text for NLP.csv', encoding='ISO-8859-2')
df.head()

#Script by Hakan Ozler https://www.kaggle.com/ozlerhakan/neural-network-word-embedding-using-keras

In [ ]:
# keras
import keras

# matplotlib
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

In [ ]:
#Remove one row where the email is null. There is NO nulls.

#df.dropna(inplace=True)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

def tokenizer_sequences(num_words, X):
    
    # when calling the texts_to_sequences method, only the top num_words are considered while
    # the Tokenizer stores everything in the word_index during fit_on_texts
    tokenizer = Tokenizer(num_words=num_words)
    # From doc: By default, all punctuation is removed, turning the texts into space-separated sequences of words
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)
    
    return tokenizer, sequences

In [ ]:
max_words = 10000 
# for the tokenizer, we configure it to only take into account the 1000 most common words when calling the texts_to_sequences method.

maxlen = 300
# maxlen is the dimension that each email will have a fixed word sequence, in this case each email will be of a 1-d tensor (300,).

In [ ]:
#Use a Tokenizer class of Keras to convert Messages text to sequences consistently

tokenizer, sequences = tokenizer_sequences(max_words, df.MESSAGE.copy())

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# # We will pad all input sequences to have the length of 300. Each email will be the same length of sequence.
X = pad_sequences(sequences, maxlen=maxlen)

y = df.CATEGORY.copy()

print('Shape of data tensor:', X.shape)
print('Shape of label tensor:', y.shape)

In [ ]:
max_words = len(tokenizer.word_index) + 1 # 33672 + 1
# 0 is reserved for padding /no data. The word indexes (i.e. tokenizer.word_index) are 1-offset.
# max_words is the size of the vocabulary, you can think of a book is of max_words pages

embedding_dim = 100 # the dimension of the word dictinory, i.e. this will be 100-dimensional word vector
# you can think of a book that each page has embedding_dim words.

#The Keras Embedding Layer

In the first example we use the embedding layer to train the word embedding alone so that we can save and use it in another model later.

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()

# embedding dictionary = 33673 * 100 = 3_367_300 parameters
# we have a 33673 x 100 word vector, Embedding accepts 2D input and returns 3D output as shown in the summary
# input_length = the length of input sequences (i.e. e-mails)
model.add(Embedding(max_words, embedding_dim, input_length=maxlen)) # 33673, 100, input_length=300 = (None, 300,100)
# the activations have shape of (33673, 300, embedding_dim=100)

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(X_train, y_train, epochs=3, batch_size=32, validation_split=0.2)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', color='red', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend();

In [ ]:
model.evaluate(X_test, y_test)
# loss value & acc metrics

In [ ]:
#save the trained word vector and use it later.  I didn't get this part

model.save_weights('pre_trained_model_100D.h5')


#I didn't get the snippet above. Where can I find that model and its number (100D.h5)?

In [ ]:
# just curiosity, let's explore the shape of the trained word embedding

model.layers[0].get_weights()[0].shape

# (vocabulary len x dimension of word vector) = word vector!

#Create a new model and add the pre-trained word vector is of shape (104211, 100).

Those numbers are on the snippet above.

In [ ]:
model2 = Sequential()

model2.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model2.add(Flatten())
model2.add(Dense(32, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

You should see the architecture, it uses 11,381,165parameters, of which 10,421,100 (the word embeddings) are non-trainable, and the remaining are 960,032 + 33. Because our vocabulary size has 104,211 words (with valid indices from 0 to 104,211) there are 104,211*100 = 10,421,100 non-trainable parameters.

The model2 assumes that the pre-trained vocabulary is of shape (104211, 100), otherwise we receive the not compatible with provided weight shape error. Since we load the embedding from outside, and do not want it to be trained, we freeze the first layer as follows:

In [ ]:
model2.layers[0].set_weights(model.layers[0].get_weights()) # load
model2.layers[0].trainable = False # freeze

In [ ]:
model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history2 = model2.fit(X_train, y_train, epochs=2, batch_size=32, validation_split=0.2)

In [ ]:
from sklearn.metrics import precision_score, recall_score

y_test_pred = np.where(model2.predict(X_test) > .5, 1, 0).reshape(1, -1)[0]

In [ ]:
print("Precision: {:.2f}%".format(100 * precision_score(y_test, y_test_pred)))
print("Recall: {:.2f}%".format(100 * recall_score(y_test, y_test_pred)))

In [ ]:
acc = history2.history['acc']
val_acc = history2.history['val_acc']
loss = history2.history['loss']
val_loss = history2.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', color='red', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend();

In [ ]:
# predict the test score
model2.evaluate(X_test, y_test)
# loss value & acc metrics

#Save the model2 along with the fully connected layers and word vector.

In [ ]:
model2.save_weights('pre_trained_model2100D_dense.h5')

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Thanks to Hakan Ozler, all code by Hakan.')

#https://www.kaggle.com/ozlerhakan/neural-network-word-embedding-using-keras